In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
mess = pd.read_csv("mess_data.csv")
mess

,dinnerate,mess,meal,excellent,good,average,poor,day,remarks
0,21-09-2023,2,dinner,4,21,94,107,Thursday,NaN
1,26-09-2023,2,lunch,45,91,37,6,Tuesday,NaN
2,26-09-2023,1,lunch,0,19,82,44,Tuesday,NaN
3,26-09-2023,2,dinner,0,20,81,45,Tuesday,NaN
4,26-09-2023,1,dinner,6,24,29,57,Tuesday,NaN
...,...,...,...,...,...,...,...,...,...
322,29-04-2024,1,lunch,1,35,34,14,Monday,NaN
323,29-04-2024,2,breakfast,1,0,6,92,Monday,NaN
324,29-04-2024,1,breakfast,2,5,7,92,Monday,NaN
325,02-05-2024,2,dinner,0,0,9,88,Thursday,NaN
